In [ ]:
import os
import string
from dotenv import load_dotenv

from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import TokenTextSplitter
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.pdf import PyPDFLoader

In [ ]:
load_dotenv(override=True)

os.environ.get("OPENAI_API_KEY")

In [ ]:
texto_completo = """
Python é uma linguagem de programação de alto nível conhecida por sua simplicidade, 
legibilidade e versatilidade. Seu design foi criado com o objetivo de ser fácil de 
aprender e usar, permitindo que programadores escrevam código de maneira mais 
intuitiva e eficiente. Ao contrário de outras linguagens, Python prioriza a 
legibilidade do código, o que facilita a compreensão e manutenção do software, 
mesmo por programadores que não são os autores do código original. 
Sua sintaxe clara, por exemplo, elimina a necessidade de muitos símbolos ou 
palavras-chave complicadas, tornando o código mais próximo da linguagem humana.
"""

In [ ]:
chunk_size = 50
chunk_overlap = 0
char_split = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
texto = "".join(f"{string.ascii_lowercase}" for _ in range(5))

In [ ]:
len(texto)

In [ ]:
texto

In [ ]:
split = char_split.split_text(texto)

In [ ]:
len(split)

In [ ]:
split

In [ ]:
char_split = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=""
)

In [ ]:
split = char_split.split_text(texto)

In [ ]:
len(split)

In [ ]:
split

In [ ]:
chunk_size = 50
chunk_overlap = 0
char_split = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator=""
)

In [ ]:
split = char_split.split_text(texto_completo)

In [ ]:
len(split)

In [ ]:
split

# TextSplit Recursive

In [ ]:
chunk_size = 50
chunk_overlap = 10
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=[".", " ", ""]
)

In [ ]:
split = char_split.split_text(texto)

In [ ]:
len(split)

In [ ]:
split

In [ ]:
chunk_size = 250
chunk_overlap = 10
char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n","\n", " ", ""]
)

In [ ]:
split = char_split.split_text(texto_completo)

In [ ]:
len(split)

In [ ]:
split

# TextSplit Tokenizer

In [ ]:
chunk_size = 50
chunk_overlap = 5
char_split = TokenTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
split = char_split.split_text(texto)

In [ ]:
len(split)

In [ ]:
split

# TextSplit Recursive PDFLoader

In [ ]:
chunk_size = 50
chunk_overlap = 10

char_split = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
file = "D:\Visual Code\RAG\pdf\entrecomp.pdf"
loader = PyPDFLoader(file)
document = loader.load()

In [ ]:
split = char_split.split_documents(document)

In [ ]:
len(split)